In [ ]:
!pip install --upgrade google-cloud-bigquery

from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Аутентифікація
auth.authenticate_user()

# Створення клієнта для BigQuery
client = bigquery.Client(project="data-analytics-mate")

# SQL-запит
query = """
with session_info as(
SELECT
  s.date,
  s.ga_session_id,
  sp.country,
  sp.device,
  sp.continent,
  sp.channel,
  ab.test,
  ab.test_group
FROM `data-analytics-mate.DA.ab_test` ab
join `data-analytics-mate.DA.session` s
on ab.ga_session_id = s.ga_session_id
join `data-analytics-mate.DA.session_params` sp
on sp.ga_session_id = ab.ga_session_id
),
-- Створення CTE "session_with_orders":
-- Підрахунок кількості сесій, які призвели до замовлення.
-- Дані групуються за атрибутами сесії (дата, країна, пристрій тощо).
session_with_orders as(
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  count(distinct o.ga_session_id) as session_with_orders
FROM `data-analytics-mate.DA.order` o
join session_info
on o.ga_session_id = session_info.ga_session_id
group by
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
  ),
-- Створення CTE "events":
-- Підрахунок кількості подій для кожної сесії.
-- Події групуються за атрибутами сесії (дата, країна, пристрій тощо) і типом події.
events as (


SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  sp.event_name,
  count(sp.ga_session_id) as event_cnt
FROM `data-analytics-mate.DA.event_params` sp
join session_info
on sp.ga_session_id = session_info.ga_session_id
group by
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  sp.event_name
),
-- Створення CTE "session":
-- Підрахунок загальної кількості сесій.
-- Дані групуються за атрибутами сесії (дата, країна, пристрій тощо).
session as (
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  count(distinct session_info.ga_session_id) as session_cnt
FROM session_info
group by
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
),
-- Створення CTE "account":
-- Підрахунок кількості нових акаунтів, створених під час сесій.
-- Дані групуються за атрибутами сесії (дата, країна, пристрій тощо).
account as (
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  count(distinct acs.ga_session_id ) as new_account_cnt
FROM `data-analytics-mate.DA.account_session` acs
join session_info
on acs.ga_session_id = session_info.ga_session_id
group by
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
)
-- Основний SELECT-запит:
-- Поєднання даних із усіх створених CTE через UNION ALL.
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  'session with orders' as event_name,
  session_with_orders.session_with_orders as value
from session_with_orders
union all
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  event_cnt as value
from events
union all
SELECT
  session.date,
  session.country,
  session.device,
  session.continent,
  session.channel,
  session.test,
  session.test_group,
  'session' as event_name,
  session_cnt as value
from session
union all
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  'new account' as event_name,
  new_account_cnt as value
from account

"""

# Виконання запиту
query_job = client.query(query)  # Виконання SQL-запиту
results = query_job.result()  # Очікування завершення запиту

# Перетворення результатів на DataFrame
df = results.to_dataframe()

# Виведення результату
df.head()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.25.0
    Uninstalling google-cloud-bigquery-3.25.0:
      Successfully uninstalled google-cloud-bigquery-3.25.0


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Функція для розрахунку Z-теста
def calculate_z_test(numerator_control, denominator_control, numerator_experiment, denominator_experiment):
    """
    Розрахунок Z-теста для статистичної значимості різниці між пропорціями
    контрольної та експериментальної груп.

    Параметри:
    - numerator_control: чисельник для контрольної групи (кількість успішних подій).
    - denominator_control: знаменник для контрольної групи (загальна кількість сесій).
    - numerator_experiment: чисельник для експериментальної групи (кількість успішних подій).
    - denominator_experiment: знаменник для експериментальної групи (загальна кількість сесій).

    Повертає:
    - z_stat: Z-статистика для тестування гіпотези.
    - p_value: ймовірність (p-значення) отримання результату випадково.
    """
    prop_control = numerator_control / denominator_control  # розрахунок пропорції для контрольної групи
    prop_experiment = numerator_experiment / denominator_experiment  # розрахунок пропорції для експериментальної групи
    pooled_prop = (numerator_control + numerator_experiment) / (denominator_control + denominator_experiment)  # обчислення зваженого середнього
    pooled_variance = pooled_prop * (1 - pooled_prop) * (1/denominator_control + 1/denominator_experiment)  # обчислення дисперсії
    z_stat = (prop_experiment - prop_control) / np.sqrt(pooled_variance)  # обчислення Z-статистики
    p_value = 2 * (1 - norm.cdf(abs(z_stat)))  # розрахунок p-значення
    return z_stat, p_value

# Функція для розрахунку статистики по всім метрикам
def calculate_statistics(data, metrics):
    """
    Розрахунок статистики для кожної метрики по всім тестам.

    Параметри:
    - data: DataFrame з даними, які містять інформацію про сесії, події та тестування.
    - metrics: словник метрик для аналізу (відображення "метрика у коді" -> "метрика у даних").

    Повертає:
    - DataFrame з результатами статистичного аналізу.
    """
    results = []
    for metric_code, metric_data_name in metrics.items():
        for test in data['test'].unique():
            # Фільтрація даних для поточного тесту
            test_data = data[data['test'] == test]
            control_data = test_data[test_data['test_group'] == 1]
            experiment_data = test_data[test_data['test_group'] == 2]

            # Розрахунок чисельників і знаменників для контрольної групи
            numerator_control = control_data.loc[control_data['event_name'] == metric_data_name, 'value'].sum()
            denominator_control = control_data.loc[control_data['event_name'] == 'session', 'value'].sum()

            # Розрахунок чисельників і знаменників для експериментальної групи
            numerator_experiment = experiment_data.loc[experiment_data['event_name'] == metric_data_name, 'value'].sum()
            denominator_experiment = experiment_data.loc[experiment_data['event_name'] == 'session', 'value'].sum()

            # Перевірка на нульовий знаменник
            if denominator_control > 0 and denominator_experiment > 0:
                conversion_rate_control = numerator_control / denominator_control  # розрахунок конверсії для контрольної групи
                conversion_rate_experiment = numerator_experiment / denominator_experiment  # розрахунок конверсії для експериментальної групи
                z_stat, p_value = calculate_z_test(numerator_control, denominator_control, numerator_experiment, denominator_experiment)  # розрахунок Z-статистики та p-значення
                metric_change = (conversion_rate_experiment / conversion_rate_control) - 1  # обчислення відсоткової зміни метрики
            else:
                conversion_rate_control, conversion_rate_experiment, z_stat, p_value, metric_change = np.nan, np.nan, np.nan, np.nan, np.nan

            # Збереження результатів у таблиці
            results.append({
                'test_number': test,
                'metric': metric_code,
                'numerator_ev': numerator_experiment,
                'denominator_ev': denominator_experiment,
                'conversion_rate_ev': conversion_rate_experiment,
                'numerator_co': numerator_control,
                'denominator_co': denominator_control,
                'conversion_rate_co': conversion_rate_control,
                'metric_change': metric_change,
                'z_stat': z_stat,
                'p_value': p_value,
                'significant': p_value < 0.05 if not np.isnan(p_value) else False
            })

    return pd.DataFrame(results)

# Словник метрик (відображення "назва метрики у коді" -> "назва метрики у даних")
metric_mapping = {
    "add_payment_info": "add_payment_info",
    "add_shipping_info": "add_shipping_info",
    "begin_checkout": "begin_checkout",
    "new_accounts": "new account"  # Вказано правильне ім'я метрики з даних
}

# Розрахунок статистики
results = calculate_statistics(df, metric_mapping)

# Збереження результатів
results.to_csv("statistics_results_dashboard.csv", index=False)
print("Файл збережено як statistics_results_dashboard.csv")


Файл збережено як statistics_results_dashboard.csv


In [ ]:
from IPython.display import display

display(results)

,test_number,metric,numerator_ev,denominator_ev,conversion_rate_ev,numerator_co,denominator_co,conversion_rate_co,metric_change,z_stat,p_value,significant
0,2,add_payment_info,2409,50244,0.047946,2344,50637,0.046290,0.035769,1.240994,0.214608,False
1,1,add_payment_info,2229,45193,0.049322,1988,45362,0.043825,0.125420,3.924884,0.000087,True
2,4,add_payment_info,3601,105141,0.034249,3731,105079,0.035507,-0.035412,-1.571106,0.116158,False
3,3,add_payment_info,3697,70439,0.052485,3623,70047,0.051722,0.014746,0.643172,0.520112,False
4,2,add_shipping_info,3510,50244,0.069859,3480,50637,0.068724,0.016510,0.709557,0.477979,False
5,1,add_shipping_info,3221,45193,0.071272,3034,45362,0.066884,0.065605,2.603571,0.009226,True
6,4,add_shipping_info,4956,105141,0.047137,5128,105079,0.048801,-0.034111,-1.785795,0.074132,False
7,3,add_shipping_info,5188,70439,0.073652,5298,70047,0.075635,-0.026212,-1.413727,0.157442,False
8,2,begin_checkout,4313,50244,0.085841,4262,50637,0.084168,0.019882,0.952898,0.340642,False
9,1,begin_checkout,4021,45193,0.088974,3784,45362,0.083418,0.066606,2.978783,0.002894,True


In [ ]:
from google.colab import files
files.download('statistics_results_dashboard.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def calculate_statistics_by_category(data, metrics, category_column):
    """
    Розрахунок статистики для кожної метрики у розрізі заданої категорії.

    Параметри:
    - data: DataFrame з даними, які містять інформацію про сесії, події та тестування.
    - metrics: словник метрик для аналізу (відображення "метрика у коді" -> "метрика у даних").
    - category_column: колонка для групування (наприклад, 'device' або 'channel').

    Повертає:
    - DataFrame з результатами статистичного аналізу.
    """
    results = []
    for category in data[category_column].unique():
        # Фільтруємо дані для поточної категорії
        category_data = data[data[category_column] == category]
        for metric_code, metric_data_name in metrics.items():
            for test in category_data['test'].unique():
                # Фільтрація даних для поточного тесту
                test_data = category_data[category_data['test'] == test]
                control_data = test_data[test_data['test_group'] == 1]
                experiment_data = test_data[test_data['test_group'] == 2]

                # Розрахунок чисельників і знаменників для контрольної групи
                numerator_control = control_data.loc[control_data['event_name'] == metric_data_name, 'value'].sum()
                denominator_control = control_data.loc[control_data['event_name'] == 'session', 'value'].sum()

                # Розрахунок чисельників і знаменників для експериментальної групи
                numerator_experiment = experiment_data.loc[experiment_data['event_name'] == metric_data_name, 'value'].sum()
                denominator_experiment = experiment_data.loc[experiment_data['event_name'] == 'session', 'value'].sum()

                # Перевірка на нульовий знаменник
                if denominator_control > 0 and denominator_experiment > 0:
                    conversion_rate_control = numerator_control / denominator_control  # конверсія для контрольної групи
                    conversion_rate_experiment = numerator_experiment / denominator_experiment  # конверсія для експериментальної групи
                    z_stat, p_value = calculate_z_test(numerator_control, denominator_control, numerator_experiment, denominator_experiment)  # Z-статистика
                    metric_change = (conversion_rate_experiment / conversion_rate_control) - 1  # зміна метрики
                else:
                    conversion_rate_control, conversion_rate_experiment, z_stat, p_value, metric_change = np.nan, np.nan, np.nan, np.nan, np.nan

                # Збереження результатів у таблиці
                results.append({
                    'category': category,
                    'test_number': test,
                    'metric': metric_code,
                    'numerator_ev': numerator_experiment,
                    'denominator_ev': denominator_experiment,
                    'conversion_rate_ev': conversion_rate_experiment,
                    'numerator_co': numerator_control,
                    'denominator_co': denominator_control,
                    'conversion_rate_co': conversion_rate_control,
                    'metric_change': metric_change,
                    'z_stat': z_stat,
                    'p_value': p_value,
                    'significant': p_value < 0.05 if not np.isnan(p_value) else False
                })

    return pd.DataFrame(results)

# Розрахунок для пристроїв
device_results = calculate_statistics_by_category(df, metric_mapping, 'device')
device_results.to_csv("statistics_results_by_device.csv", index=False)
print("Результати по пристроях збережено як statistics_results_by_device.csv")

# Розрахунок для каналів пошуку
channel_results = calculate_statistics_by_category(df, metric_mapping, 'channel')
channel_results.to_csv("statistics_results_by_channel.csv", index=False)
print("Результати по каналах пошуку збережено як statistics_results_by_channel.csv")


Результати по пристроях збережено як statistics_results_by_device.csv
Результати по каналах пошуку збережено як statistics_results_by_channel.csv


In [ ]:
from IPython.display import display

display(channel_results)

,category,test_number,metric,numerator_ev,denominator_ev,conversion_rate_ev,numerator_co,denominator_co,conversion_rate_co,metric_change,z_stat,p_value,significant
0,Organic Search,2,add_payment_info,597,17428,0.034255,698,17466,0.039963,-0.142834,-2.820279,0.004798,True
1,Organic Search,1,add_payment_info,514,15631,0.032883,640,15675,0.040829,-0.194614,-3.730758,0.000191,True
2,Organic Search,4,add_payment_info,1063,37831,0.028099,1092,38033,0.028712,-0.021359,-0.508372,0.611192,False
3,Organic Search,3,add_payment_info,1098,25055,0.043824,1036,24585,0.042140,0.039964,0.924884,0.355026,False
4,Organic Search,2,add_shipping_info,1040,17428,0.059674,1130,17466,0.064697,-0.077639,-1.942671,0.052056,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Undefined,3,begin_checkout,1175,4534,0.259153,1066,4636,0.229940,0.127048,3.254797,0.001135,True
76,Undefined,2,new_accounts,308,3670,0.083924,290,3661,0.079213,0.059464,0.736742,0.461280,False
77,Undefined,1,new_accounts,247,3397,0.072711,301,3336,0.090228,-0.194137,-2.628178,0.008584,True
78,Undefined,4,new_accounts,510,5862,0.087001,484,5716,0.084675,0.027475,0.446742,0.655061,False


In [ ]:
from IPython.display import display

display(device_results)

,category,test_number,metric,numerator_ev,denominator_ev,conversion_rate_ev,numerator_co,denominator_co,conversion_rate_co,metric_change,z_stat,p_value,significant
0,mobile,2,add_payment_info,961,19756,0.048643,978,20017,0.048858,-0.004401,-0.099562,0.920692,False
1,mobile,1,add_payment_info,942,17767,0.053020,810,17896,0.045262,0.171407,3.389330,0.000701,True
2,mobile,4,add_payment_info,1501,41345,0.036304,1413,41216,0.034283,0.058964,1.573870,0.115517,False
3,mobile,3,add_payment_info,1463,27423,0.053349,1463,27404,0.053386,-0.000693,-0.019267,0.984628,False
4,mobile,2,add_shipping_info,1356,19756,0.068637,1417,20017,0.070790,-0.030406,-0.842754,0.399366,False
5,mobile,1,add_shipping_info,1256,17767,0.070693,1257,17896,0.070239,0.006459,0.167387,0.867065,False
6,mobile,4,add_shipping_info,2023,41345,0.048930,1961,41216,0.047579,0.028398,0.905776,0.365054,False
7,mobile,3,add_shipping_info,2029,27423,0.073989,2071,27404,0.075573,-0.020959,-0.704992,0.480816,False
8,mobile,2,begin_checkout,1656,19756,0.083823,1766,20017,0.088225,-0.049899,-1.565423,0.117484,False
9,mobile,1,begin_checkout,1561,17767,0.087860,1593,17896,0.089014,-0.012973,-0.384029,0.700957,False


In [ ]:
from google.colab import files
files.download('statistics_results_by_device.csv')

In [ ]:
from google.colab import files
files.download('statistics_results_by_channel.csv')

Посилання на [дашборд](https://public.tableau.com/views/ABTest_17333349673250/Significance?:language=es-ES&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)